# DSCI100 Final Group Project

## (1) Data Description:

The datasets given are these two files: players.csv and sessions.csv.

players.csv contains 197 observations of:
experience: Player’s chosen experience level, such as Pro, Veteran, Amateur, etc.
subscribe: Indicates if the player has subscribed to the email notification (True/False).
played_hours: Total hours played by the player.
gender: Gender of the player.
age: Age of the player.

sessions.csv contains 1536 (and counting) observations of:
hashedEmail: Anonymized identifier linking sessions to players.
start_time and end_time: Start and end times of each play session.
original_start_time and original_end_time: Timestamps for each session's start and end.

We plan to combine the information given from both files to calculate player metrics like session frequency, average session length, total playtime and time of day or day of week patterns. There is missing data in players.csv (individualId, organizationName) and there are outliers and inconsistencies in session.csv which will be addressed through data preprocessing, like standardizing the time formats.

## (2) Question:

Question: "What time windows are most likely to have large number of simultaneous players>"

The goal is to predict the time window when the demand for concurrent player licenses is based on past patterns. We can analyze the hour of the day as well as the day of the week that most players get on, to help forecast player demand in each time window. This can help the research team manage the licenses and server capacity effectively during peak hours.

The predictors are:

Hour of the day, Day of the week, Weekend or Weekday


We will standardize all the time variables so that it can be easily plotted and analyzed.


## (3) Exploratory Data Analysis and Visualization

To gain insight into player demand patterns, we’ll:

Aggregate data to determine the number of simultaneous players for each hourly window.
Visualize how player counts vary by hour of the day and day of the week.


In [4]:
library(tidyverse)

players_url <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
sessions_url <- "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"
download.file(players_url, destfile = "players.csv", mode = "wb")
download.file(sessions_url, destfile = "sessions.csv", mode = "wb")

players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
#format data to date month year hour minute
merged_data <- merged_data %>%
  mutate(
    start_datetime = as.POSIXct(start_time, format = "%d/%m/%Y %H:%M"),
    end_datetime = as.POSIXct(end_time, format = "%d/%m/%Y %H:%M")
  )
#make new df to show time windows
#standardaize start times to within the same hour
#count simul players during said time
time_windows <- merged_data %>%
  mutate(hour = format(start_datetime, "%Y-%m-%d %H:00:00")) %>%
  group_by(hour) %>%
  summarise(simultaneous_players = n())
#convert back to time format for plotting
time_windows <- time_windows %>%
  mutate(hour = as.POSIXct(hour, format = "%Y-%m-%d %H:%M:%S"))


ERROR: Error in eval(expr, envir, enclos): object 'merged_data' not found


In [ ]:
ggplot(time_windows, aes(x = hour, y = simultaneous_players)) +
  geom_line() +
  labs(
    title = "Simultaneous Player Counts Over Time",
    x = "Time (Hourly)",
    y = "Number of Simultaneous Players"
  )


In [ ]:
#add hour of day 
time_windows <- time_windows %>%
  mutate(hour_of_day = as.integer(format(hour, "%H")))

#calculte simul players during time of day (24 hr format)
avg_demand_by_hour <- time_windows %>%
  group_by(hour_of_day) %>%
  summarise(avg_players = mean(simultaneous_players, na.rm = TRUE))

# avg simul player vs hour of day
ggplot(avg_demand_by_hour, aes(x = hour_of_day, y = avg_players)) +
  geom_line() +
  labs(
    title = "Average Player Demand by Hour of Day",
    x = "Hour of Day",
    y = "Average Simultaneous Players"
  )



In [ ]:
#add day of week
time_windows <- time_windows %>%
  mutate(day_of_week = weekdays(hour))
# count avg player simultaneous count
avg_demand_by_day <- time_windows %>%
  group_by(day_of_week) %>%
  summarise(avg_players = mean(simultaneous_players, na.rm = TRUE))

#plot average simul players vs day of week
ggplot(avg_demand_by_day, aes(x = day_of_week, y = avg_players)) +
  geom_bar(stat = "identity") +
  labs(
    title = "Average Player Demand by Day of Week",
    x = "Day of Week",
    y = "Average Simultaneous Players"
  )


## (4) Methods and Plan

We are using linear regression as an appropriate method because of the nature of the problem. This method can capture trends and patterns based on time-related variables like hour of the day and day of the week. It is also easily interpreted and allows see the demand for licenses. Using this model can allow us to predict the expected player count in each time window. We are assuming a linear relationship between our predictors and response variable so that this model can work. We assume that the player count i0n one hour is independent of player counts in other hours. The limitation of this model and plan is the possible sudden spikes in player counts that can occur at any time(As seen in Simultaneous Player Counts over Time). The data may also be showing the same player over different time windows. This means that if there is an average of 1 player count but its all different people then different licenses are still required, but the model doesn't know that. We may test the model accuracy using RMSE on the training set, and RMSPE on the test set. We will split the data to 70% training, and use the later/ more recent data periods to test. We will use K-Fold Cross Validation on the training set. We can use this plan to forecast peak demand times and allow to accomodate all parallel players. This approach offers a clear and understandable solution, while also being flexible on tweaking it based on how well it performs.

In [ ]:
players